In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from contextlib import suppress
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import os
import warnings
from zipfile import ZipFile

from skimage.io import imread, imsave

from keras import applications
from keras import optimizers
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, classification_report

Using TensorFlow backend.


In [0]:
np.random.seed(8)

In [0]:
!pwd
%cd "drive/My Drive/CV_Project"
!pwd

/content
/content/drive/My Drive/CV_Project
/content/drive/My Drive/CV_Project


In [0]:
class_names = os.listdir('UCMerced_LandUse/Train/')
target_dirs = {target: './UCMerced_LandUse/' + target for target in ['Train', 'Validation', 'Test']}

In [0]:
print(class_names)
print(target_dirs)

['storagetanks', 'tenniscourt', 'sparseresidential', 'river', 'runway', 'parkinglot', 'overpass', 'harbor', 'intersection', 'mobilehomepark', 'mediumresidential', 'golfcourse', 'freeway', 'denseresidential', 'forest', 'buildings', 'chaparral', 'beach', 'baseballdiamond', 'airplane', 'agricultural']
{'Train': './UCMerced_LandUse/Train', 'Validation': './UCMerced_LandUse/Validation', 'Test': './UCMerced_LandUse/Test'}


In [0]:
def get_bottleneck_features(model, dataset, batch_size=64):
  print(f'Generating "{dataset}" bottleneck predictions')
  image_data_gen = ImageDataGenerator(rescale=1/255.0)
  image_generator = image_data_gen.flow_from_directory(target_dirs[dataset],
                                                        batch_size=batch_size,
                                                        shuffle=False)
  image_count = 0
  X_batches, Y_batches = [], []
  for X, Y in image_generator:
    X_batches.append(model.predict_on_batch(X))
    Y_batches.append(Y)
    image_count += X.shape[0]
    # Must interrupt image_generator
    if image_count >= image_generator.n:
      break
  X = np.concatenate(X_batches)
  Y = np.concatenate(Y_batches)
  return X, Y

In [0]:
pretrained_model = applications.VGG16(include_top=False, weights='imagenet')





58892288/58889256 [==============================] - 6s 0us/step








In [0]:
num_classes = len(class_names)
X, Y = dict(), dict()
for dataset in ['Train', 'Validation', 'Test']:
    # Extract bottleneck features from pretrained model, predicting from "dataset" directory
    X[dataset], Y[dataset] = get_bottleneck_features(pretrained_model, dataset)

Generating "Train" bottleneck predictions
Found 1785 images belonging to 21 classes.
Generating "Validation" bottleneck predictions
Found 210 images belonging to 21 classes.
Generating "Test" bottleneck predictions
Found 210 images belonging to 21 classes.


In [0]:
def build_fully_connected(input_shape, num_classes):
    """
    Create a fully-connected model to train or test on UC Merced dataset.
    """
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [0]:
model = build_fully_connected(input_shape=X['Train'].shape[1:], num_classes=num_classes)
adam = optimizers.Adam(lr=0.0001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model_fit_history = model.fit(X['Train'], Y['Train'], batch_size=64, epochs=300,
                              verbose=2, validation_data=(X['Validation'], Y['Validation']))

Train on 1785 samples, validate on 210 samples
Epoch 1/300
 - 1s - loss: 2.2836 - acc: 0.3462 - val_loss: 1.3959 - val_acc: 0.7095
Epoch 2/300
 - 0s - loss: 1.1002 - acc: 0.6947 - val_loss: 0.9539 - val_acc: 0.7619
Epoch 3/300
 - 0s - loss: 0.6933 - acc: 0.8129 - val_loss: 0.7961 - val_acc: 0.7571
Epoch 4/300
 - 0s - loss: 0.4999 - acc: 0.8644 - val_loss: 0.7093 - val_acc: 0.7667
Epoch 5/300
 - 0s - loss: 0.3630 - acc: 0.9143 - val_loss: 0.6479 - val_acc: 0.7905
Epoch 6/300
 - 0s - loss: 0.2844 - acc: 0.9412 - val_loss: 0.6392 - val_acc: 0.7762
Epoch 7/300
 - 0s - loss: 0.2391 - acc: 0.9462 - val_loss: 0.5994 - val_acc: 0.8143
Epoch 8/300
 - 0s - loss: 0.1992 - acc: 0.9625 - val_loss: 0.5929 - val_acc: 0.8000
Epoch 9/300
 - 1s - loss: 0.1619 - acc: 0.9737 - val_loss: 0.5480 - val_acc: 0.8048
Epoch 10/300
 - 0s - loss: 0.1384 - acc: 0.9742 - val_loss: 0.5720 - val_acc: 0.8000
Epoch 11/300
 - 0s - loss: 0.1222 - acc: 0.9798 - val_loss: 0.5704 - val_acc: 0.8048
Epoch 12/300
 - 0s - loss: 

In [0]:
epochs = np.argmin(model_fit_history.history['val_loss']) + 1
print(f'Stop training at {epochs} epochs')

# Merge training and validation data
X_train = np.concatenate([X['Train'], X['Validation']])
Y_train = np.concatenate([Y['Train'], Y['Validation']])

# Randomly shuffle X and Y
shuffle_index = np.random.permutation(len(X_train))
X_train = X_train[shuffle_index]
Y_train = Y_train[shuffle_index]
model = build_fully_connected(input_shape=X_train.shape[1:], num_classes=num_classes)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model_fit_history = model.fit(X_train, Y_train, batch_size=64, epochs=epochs, verbose=0)

NameError: ignored

In [0]:
y_pred = model.predict_classes(X['Test'], verbose=0)

In [0]:
y_test = np.nonzero(Y['Test'])[1]
accuracy = accuracy_score(y_test, y_pred)
print(f'Model predication accuracy: {accuracy:.3f}')
print(f'\nClassification report:\n {classification_report(y_test, y_pred)}')

Model predication accuracy: 0.886

Classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.80      0.89        10
           2       1.00      0.90      0.95        10
           3       0.69      0.90      0.78        10
           4       0.75      0.90      0.82        10
           5       1.00      0.80      0.89        10
           6       1.00      0.30      0.46        10
           7       0.77      1.00      0.87        10
           8       1.00      0.80      0.89        10
           9       1.00      0.90      0.95        10
          10       1.00      1.00      1.00        10
          11       0.75      0.90      0.82        10
          12       0.69      0.90      0.78        10
          13       0.91      1.00      0.95        10
          14       0.90      0.90      0.90        10
          15       1.00      1.00      1.00        10
          16       0.8